## 前置动作
1. 克隆`https://github.com/unit-mesh/unit-minions`项目
1. 将`userstory_detail_double_clean_cn.jsonl`数据集放到`input1`或者修改下面的数据集路径
1. 将`ChatGLM-6b`的模型文件准备到`input2`或者修改下面模型引用的路径。或者直接指定为`THUDM/chatglm-6b`
1. 如果不是用的`openbayes`平台，记得修改下方的模型输出路径，并提前创建好路径

In [ ]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git

In [5]:
%cd ChatGLM-Tuning
pip install -r requirements.txt 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-_fs4her9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-_fs4her9
  Resolved https://github.com/huggingface/peft.git to commit 1117d4772109a098787ce7fc297cb6cd641de6eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 1.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 505.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/4

In [3]:
!python cover_alpaca2jsonl.py \
    --data_path /openbayes/input/input0/userstory_detail_double_clean_cn.jsonl \
    --save_path /output/dataset/userstory_detail_double_clean_cn_alpaca.jsonl

formatting..: 100%|██████████████████████| 2462/2462 [00:00<00:00, 72707.65it/s]


In [4]:
!python tokenize_dataset_rows.py \
    --jsonl_path /openbayes/home/dataset/userstory_detail_double_clean_cn_alpaca.jsonl \
    --save_path /openbayes/home/dataset \
    --max_seq_length 380 \
    --model_name /openbayes/input/input1

Found cached dataset generator (/root/.cache/huggingface/datasets/generator/default-0133ef0c8080d865/0.0.0)


In [5]:
import sys

sys.path.append("../")

In [6]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

model = AutoModel.from_pretrained("/openbayes/input/input1", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

model.is_parallelizable = True
model.model_parallel = True


/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tcp'), PosixPath('443'), PosixPath('//10.111.0.1')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//alchemist-experience'), PosixPath('7890')}
  warn(msg)
/usr/loc


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


## Test before finetune

In [8]:
from cover_alpaca2jsonl import format_example
import json


with open("/openbayes/input/input0/userstory_detail_double_clean_cn.jsonl", encoding="utf-8") as f:
    examples = list(f)
    examplesSample = examples[:2]

# instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(examplesSample):
        item = json.loads(item)
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/usr/local/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The dtype of attention mask (torch.int64) is not bool


Instruction: create agile user story for following topic
Input: animation and comics:browse and search for animations and comics
Answer: User Story:

Title: browse and search for animations and Comics

Objective: Develop a user-friendly interface for browsing and searching for animations and Comics

Context: The company is looking to create a new user interface for their animation and漫画 collection. The user interface will be used to browse and search for animations and漫画, as well as to access specific titles and章节.

Action steps:

1.  Create a user manual and a test user interface to evaluate the effectiveness of the interface.
2.  Implement the user manual and test the interface with a small
### 1.Answer:
 用户故事：可以浏览和搜索动画和漫画
作为一个 animation and comics 应用的用户
我想要浏览和搜索动画和漫画
以便于我能够找到我喜欢的动画和漫画

ac 1: 用户可以浏览和搜索动画和漫画
假设 用户打开 animation and comics 应用
当 用户点击浏览和搜索动画和漫画按钮
于是 用户可以看到动画和漫画的列表，并可以搜索特定的动画和漫画 


Instruction: create agile user story for following topic
Input: animation and comics:particip

In [9]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [10]:
import datasets

dataset_path = "/openbayes/home/dataset"

dataset = datasets.load_from_disk(dataset_path)

# train_num = 2000

# mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])
mini_train_dataset = dataset

In [11]:
from transformers import Trainer, HfArgumentParser
import os


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss
    
    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME

        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

In [12]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 8,
    learning_rate = 1e-4,
    max_steps=3000,
    save_steps=1000,
    logging_steps=100,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

/usr/local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
100,0.986000
200,0.572900
300,0.532600
400,0.470700
500,0.453300
600,0.434100
700,0.388400
800,0.372100
900,0.384300
1000,0.336400


/usr/local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=3000, training_loss=0.30902166016896565, metrics={'train_runtime': 5566.3363, 'train_samples_per_second': 4.312, 'train_steps_per_second': 0.539, 'total_flos': 1.516394609347461e+17, 'train_loss': 0.30902166016896565, 'epoch': 9.74})

## Test After finetune

In [13]:
from cover_alpaca2jsonl import format_example
import json


# instructions = json.load(open("data/alpaca_data.json"))


# with torch.no_grad():
#     for idx, item in enumerate(instructions[:5]):
#         feature = format_example(item)
#         input_text = feature["context"]
#         input_ids = tokenizer.encode(input_text, return_tensors='pt')
#         out = model.generate(
#             input_ids=input_ids,
#             max_length=150,
#             temperature=0
#         )
#         answer = tokenizer.decode(out[0])
#         print(answer)
#         item['infer_answer'] = answer
#         print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

# from cover_alpaca2jsonl import format_example
# import json


with open("/openbayes/input/input0/userstory_detail_double_clean_cn.jsonl", encoding="utf-8") as f:
    examples = list(f)
    examplesSample = examples[:2]
    
    

# instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(examplesSample):
        item = json.loads(item)
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/usr/local/lib/python3.8/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Instruction: create agile user story for following topic
Input: animation and comics:browse and search for animations and comics
Answer: 用户故事:可以浏览和搜索动画和漫画
作为一个 animation and comics 应用的用户
我想要浏览和搜索动画和漫画
以便于我能够找到我喜欢的动画和漫画

ac 1: 用户可以浏览和搜索动画和漫画
假设 用户打开 animation and comics 应用
当 用户点击浏览和搜索动画和漫画按钮
于是 用户可以看到动画和漫画的列表,并可以搜索特定的动画和漫画
### 1.Answer:
 用户故事：可以浏览和搜索动画和漫画
作为一个 animation and comics 应用的用户
我想要浏览和搜索动画和漫画
以便于我能够找到我喜欢的动画和漫画

ac 1: 用户可以浏览和搜索动画和漫画
假设 用户打开 animation and comics 应用
当 用户点击浏览和搜索动画和漫画按钮
于是 用户可以看到动画和漫画的列表，并可以搜索特定的动画和漫画 


Instruction: create agile user story for following topic
Input: animation and comics:participate in online forums and discussions
Answer: 用户故事:可以参与在线论坛和讨论
作为一个 animation and comics 应用的用户
我想参与在线论坛和讨论
以便于我可以与其他用户分享我的作品,讨论漫画,以及讨论其他主题。

ac 1: 用户可以参与在线论坛和讨论
ac 2: 用户可以在论坛和讨论中发表作品
ac 3: 用户可以在论坛和讨论中讨论漫画,讨论其他主题,以及分享其他内容
### 2.Answer:
 用户故事：可以参与在线论坛和讨论
作为一个 animation and comics 应用的用户
我想参与在线论坛和讨论
以便于我可以与其他爱好者分享我的想法，观点和经验。

ac 1: 用户可以参与在线论坛和讨论
假设 用户已经登录到 animation and comics 应用


In [14]:
import os

model.save_pretrained("/output/models/lora/chatglm/userstory_detail")

def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)
save_tunable_parameters(model, os.path.join("/output/models/lora/chatglm/userstory_detail", "chatglm-lora-userstory_detail.pt"))

In [2]:
%cd ..
pip install -r unit-minions/apps/userstory/chatglm/chatglm-web/requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 117.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 142.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 132.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.4/271.4 kB 80.8 MB/s eta 0:00:00

In [ ]:
!python unit-minions/apps/userstory/chatglm/chatglm-web/app.py


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('home/fadbaa56-d92a-4147-b576-6b27bfacd360')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.